In [ ]:
!pip install sentence-transformers

In [ ]:
!pip install datasets

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample, losses, util
from torch.utils.data import DataLoader
import json
import pandas as pd
import torch
import os
import itertools

In [ ]:
json_data='./path/to/json/file.json'

with open(json_data, 'r') as f:
    data = json.load(f)

In [ ]:
similar_questions = ["Is it possible to create a joint general investment account?",
"Do you provide an option for a joint investment account?",
"What are the minimum funds required to maintain an account?",
"Is there a specific minimum balance needed to open an account?",
"When will I receive a summary of my taxes?",
"How do I access a yearly tax summary for my investment account?",
"How can I view the history of my transactions?",
"Where can I check all my past transactions in the app?",
"What are the benefits of a flexible stocks & shares ISA?",
"Can you explain what a flexible stocks & shares ISA is?",
"Why is my national insurance number required for an ISA?",
"Why do I need to provide my NI number to open a stocks & shares ISA?",
"How many ISAs can I own simultaneously?",
"Is it possible to hold multiple stocks & shares ISAs?",
"How much money do I need to get started with an ISA?",
"What is the minimum amount required to open an ISA?",
"Can I transfer my existing ISA to a new provider?",
"What is the process for transferring a stocks & shares ISA to another provider?",
"Can I withdraw cash from my ISA at any time?",
"Is it possible to withdraw money from my stocks & shares ISA?",
"Why does my ISA allowance become negative when I withdraw funds?",
"Can you explain why my ISA shows a negative balance after withdrawal?",
"Do you offer junior ISAs or cash ISAs?",
"Is there a plan to offer lifetime ISAs or other types?",
"What exactly is a self-invested personal pension (SIPP)?",
"Could you describe the key features of a SIPP?",
"Who would benefit the most from having a SIPP?",
"What kind of investor is a SIPP best suited for?",
"Who administers the CMC Invest SIPP?",
"Which company manages the CMC Invest SIPP?",
"Can I hold more than one pension plan at a time?",
"Is it possible to contribute to more than one pension simultaneously?",
"What tax relief is available on pension contributions?",
"What are the available tax benefits for pension contributions?",
"How much tax relief can I claim on my pension contributions?",
"What is the limit for tax relief on pensions?",
"How do I add money to my CMC Invest SIPP?",
"What are the methods for contributing to my CMC Invest SIPP?",
"How does tax relief work for a SIPP with CMC Invest?",
"What are the tax relief procedures when contributing to a SIPP?",
"Can I transfer my existing pension into a CMC Invest SIPP?",
"How can I transfer my current pension plan to a CMC Invest SIPP?",
"Can I move my SIPP to another pension provider?",
"Is it possible to transfer my SIPP to a different pension scheme?",
"What kinds of assets can I invest in through my SIPP?",
"Which investment options are available for my SIPP?",
"Is there a cap on how much I can accumulate in a pension?",
"What are the pension accumulation limits?",
"How much does it cost to maintain a CMC Invest SIPP?",
"What are the associated fees for the CMC Invest SIPP?",
"When can I start withdrawing from my SIPP?",
"At what age can I begin taking money from my SIPP?",
"Can I access my pension before the age of 55?",
"Is it possible to take my pension early under certain conditions?",
"What happens to my SIPP if I pass away?",
"How are my SIPP funds handled in case of death?",
"How do I purchase shares through the app?",
"Can you guide me on how to buy shares in the app for the first time?",
"Is the price I see guaranteed when placing a share order?",
"Are share prices final when I confirm my order?",
"How are foreign exchange (FX) fees calculated?",
"What is the process for calculating FX fees?",
"What does 'settlement' mean in relation to buying and selling shares?",
"How does the settlement process work for my investments?",
"Do I have to pay stamp duty on UK shares?",
"Is there stamp duty on purchasing shares in the UK?",
"Is stamp duty applicable to US shares?",
"Do I need to pay taxes on US shares?",
"What is an exchange-traded fund (ETF)?",
"How do ETFs differ from regular shares?",
"What exactly is an investment trust?",
"Can you explain what an investment trust is?",
"Are there any charges for investing in ETFs or investment trusts?",
"Do I have to pay fees when investing in ETFs or trusts?",
"What is a mutual fund?",
"How does a mutual fund work compared to other investment types?",
"Why should I consider investing in a mutual fund?",
"What are the advantages of investing in mutual funds?",
"How is income from mutual funds distributed?",
"What are the options for receiving income from mutual funds?",
"How do I locate the income class for a mutual fund?",
"Where can I find details about mutual fund classes in the app?",
"Are there any fees associated with mutual fund investments?",
"What are the charges for investing in mutual funds through the app?",
"Will more mutual funds be added to the app?",
"Is there a plan to expand the selection of mutual funds available?",
"How do I invest in mutual funds on the app?",
"Can you walk me through the process of investing in mutual funds?",
"Can I cancel a pending mutual fund order?",
"Is it possible to stop a mutual fund order after placing it?",
"Are limit orders available for mutual fund investments?",
"Can I place limit orders when buying or selling mutual funds?",
"How do I add money to my investment account?",
"What are the methods for depositing money into my account?",
"How do I withdraw money from my account?",
"What steps do I need to follow to withdraw funds?",
"How long does it take for a withdrawal to process?",
"What is the processing time for withdrawals?",
"Why is some of my cash available for investment but not for withdrawal?",
"Can I withdraw unsettled cash from my account?"]

In [ ]:
irrelevant_financial_questions = ["How does compound interest work, and why is it important in investing?",
"What is a certificate of deposit (CD), and how does it compare to a savings account?",
"How can I create a diversified investment portfolio?",
"What are the different types of bonds, and how do they work?",
"How do I invest in government bonds?",
"What is the difference between preferred stock and common stock?",
"How do dividend reinvestment plans (DRIPs) work?",
"What is the difference between passive and active investing?",
"What is dollar-cost averaging, and how can it benefit my investments?",
"How can I start investing in real estate?",
"What is the difference between a traditional IRA and a Roth IRA?",
"How can I reduce my taxable income through retirement contributions?",
"What are index funds, and how do they differ from mutual funds?",
"What are the advantages and disadvantages of ETFs?",
"What is a hedge fund, and how does it differ from a mutual fund?",
"What is an options contract, and how does it work?",
"How can I invest in commodities like gold and oil?",
"What are the risks of investing in cryptocurrency?",
"What is a margin account, and how does it work?",
"How can I protect my investments from inflation?",
"What is a stock split, and how does it affect my shares?",
"What are REITs (real estate investment trusts), and how can I invest in them?",
"How does short selling work in the stock market?",
"What is the role of a financial advisor, and when should I hire one?",
"What is a 401(k) plan, and how does it help in retirement savings?",
"How do I calculate my net worth?",
"What is credit risk, and how does it affect bond investments?",
"What is a stock buyback, and how does it impact shareholders?",
"How can I invest in foreign stocks?",
"What are the tax implications of selling investments?",
"How do I use a stock screener to find investment opportunities?",
"What are the different types of mutual funds, and how do they differ?",
"How do I set financial goals for retirement?",
"What is the difference between growth stocks and value stocks?",
"How can I minimize fees and costs when investing?",
"What is a corporate bond, and how does it differ from a government bond?",
"What are the benefits of socially responsible investing?",
"How do currency exchange rates affect international investments?",
"What are penny stocks, and are they a good investment?",
"What is a capital gains tax, and how does it work?",
"How can I build a budget for my personal finances?",
"What is a financial ratio, and how is it used in stock analysis?",
"How do I rebalance my investment portfolio?",
"What is liquidity, and why is it important in investing?",
"How do I determine my risk tolerance for investing?",
"What is the time value of money, and how is it calculated?",
"How can I use tax-advantaged accounts to grow my savings?",
"What are target-date funds, and how do they work?",
"How does an investment in foreign exchange (Forex) work?",
"What are the differences between different types of annuities?",
"How can I use leverage in investing, and what are the risks?",
"What is an IPO (initial public offering), and how can I invest in one?",
"How do I evaluate a company’s financial statements before investing?",
"What is a credit score, and how does it affect my ability to get a loan?",
"How can I improve my credit score?",
"What is the difference between secured and unsecured loans?",
"How does refinancing a mortgage work?",
"What are the risks and rewards of investing in emerging markets?",
"What is inflation, and how does it affect my purchasing power?",
"How do I manage financial risk in my investment portfolio?",
"What is a financial bubble, and how can I recognize one?",
"How can I save for my child’s college education?",
"What are treasury bills, and how do they work?",
"What is the debt-to-equity ratio, and why is it important?",
"How do I calculate the return on investment (ROI)?",
"What is a dividend yield, and how is it calculated?",
"How can I start investing with little money?",
"What is a wealth management service, and how can it benefit me?",
"What is a structured note, and how does it work?",
"What are derivatives, and how are they used in investing?",
"How do I evaluate a company’s price-to-earnings (P/E) ratio?",
"What are municipal bonds, and how do they differ from corporate bonds?",
"How can I hedge against currency fluctuations?",
"What is the difference between a traditional and Roth 401(k)?",
"How does an economic recession affect my investments?",
"What are the key indicators of a healthy economy?",
"What is a stop-loss order, and how does it work in trading?",
"How can I invest in private equity?",
"What is a crowdfunding investment, and how does it work?",
"What are the risks associated with venture capital investing?",
"How do tax credits differ from tax deductions?",
"How can I invest in small businesses?",
"What is an income statement, and how is it used in financial analysis?",
"How does the Federal Reserve influence interest rates and the economy?",
"What are the benefits of investing in index funds over individual stocks?",
"How do I calculate the yield to maturity of a bond?",
"What is a financial advisor's fiduciary duty?",
"How can I prepare financially for an economic downturn?",
"What is the difference between quantitative and qualitative analysis in finance?",
"What are growth mutual funds, and how do they differ from income mutual funds?",
"How can I invest in green energy?",
"What is the role of liquidity in financial markets?",
"How do interest rates impact bond prices?",
"What is diversification, and why is it important in investing?",
"How do margin calls work in trading?",
"What are credit default swaps, and how are they used in the financial industry?",
"How does a company go public, and what is the role of an underwriter?",
"What is the difference between a financial planner and a financial advisor?",
"How can I invest in artificial intelligence (AI) technologies?"]

In [ ]:
irrelevant_questions = ["What is the capital city of Canada?",
"How do airplanes stay in the sky?",
"What is the boiling point of water at sea level?",
"Who painted the Mona Lisa?",
"What are the primary ingredients in a margherita pizza?",
"How do you change a flat tire?",
"What is the speed of light in a vacuum?",
"Who wrote 'Pride and Prejudice'?",
"What is the chemical formula for table salt?",
"How does a camera work?",
"What are the seven continents of the world?",
"How do plants perform photosynthesis?",
"What is the largest mammal on Earth?",
"What is the difference between an alligator and a crocodile?",
"How many bones are in the human body?",
"What is the longest river in the world?",
"What is the primary function of red blood cells?",
"Who discovered penicillin?",
"What are the three primary colors?",
"How many planets are in the solar system?",
"What is the main ingredient in chocolate?",
"What is the hottest planet in our solar system?",
"How long is a marathon?",
"What is the process of mitosis?",
"How does a refrigerator work?",
"Where is the Great Barrier Reef located?",
"Who was the first person to walk on the moon?",
"What is the tallest mountain in the world?",
"What does DNA stand for?",
"What is the smallest unit of matter?",
"What is a black hole?",
"How does a microwave oven heat food?",
"What is the hardest natural substance on Earth?",
"What is the fastest land animal?",
"What is the purpose of the United Nations?",
"What is a haiku?",
"How do bees make honey?",
"What is the capital of Australia?",
"What are the three states of matter?",
"What is the longest bone in the human body?",
"What is the deepest ocean in the world?",
"How do magnets work?",
"What is the most widely spoken language in the world?",
"What is the chemical formula for water?",
"How many hearts does an octopus have?",
"Who invented the telephone?",
"What is the Pythagorean theorem?",
"What is the lifespan of a housefly?",
"What causes a rainbow?",
"What is the square root of 64?",
"How many sides does a hexagon have?",
"Where is the Taj Mahal located?",
"How does the human brain process information?",
"What is the function of the immune system?",
"What are Newton’s three laws of motion?",
"What is the largest desert on Earth?",
"How does a compass work?",
"Who wrote 'To Kill a Mockingbird'?",
"What is the main ingredient in bread?",
"Where do penguins live?",
"What is a leap year?",
"How do volcanoes erupt?",
"What is the primary function of the heart?",
"Who painted the ceiling of the Sistine Chapel?",
"What is the chemical symbol for gold?",
"How do airplanes generate lift?",
"What are the stages of the water cycle?",
"What is the difference between weather and climate?",
"Who was the first female astronaut?",
"What is the primary function of the kidneys?",
"What is the freezing point of water?",
"How do plants grow?",
"What is the capital of France?",
"What is the function of chlorophyll in plants?",
"How does a wind turbine generate electricity?",
"What are the five senses?",
"What is the largest country in the world by land area?",
"How do birds fly?",
"What is the definition of gravity?",
"How does a light bulb work?",
"What is the primary function of the lungs?",
"Who discovered gravity?",
"What is the Great Wall of China?",
"What is the fastest fish in the ocean?",
"What is the capital of Italy?",
"How does a rocket work?",
"What are the layers of the Earth?",
"What is a solar eclipse?",
"How does the internet work?",
"What is the largest organ in the human body?",
"What are the basic ingredients of soap?",
"Who invented the printing press?",
"What is the purpose of the ozone layer?",
"What is the capital of Japan?",
"How does a satellite orbit the Earth?",
"How many legs does a spider have?",
"Who composed Beethoven’s 9th Symphony?",
"What is the tallest tree in the world?",
"What are the phases of the moon?",
"What is the meaning of photosynthesis?",
"What are the five oceans of the world?"]

In [ ]:
# Extract articles from the JSON data
articles = []
for topic in data:
    for article in topic['articles']:
        articles.append({
            'id': article['id'],
            'topicId': topic['id'],
            'title': article['title'],
            'body': article['body']
        })

questions = [faq['title'] for faq in articles]
answers = [faq['body'] for faq in articles]

df = pd.DataFrame(articles)
df

In [ ]:
def retrieve_top_k_answers(user_query, k=1):

  query_embedding = model.encode(user_query, convert_to_tensor=True)
  # question_embeddings = model.encode(questions, convert_to_tensor=True)

  cosine_scores = util.cos_sim(query_embedding, question_embeddings)
  top_k = torch.topk(cosine_scores, k=k)

  top_k_answers = []
  for idx, score in zip(top_k.indices[0], top_k.values[0]):
        # Convert tensor to integer index
        idx = idx.item()
        score = score.item()

        top_k_answers.append({
            'question': questions[idx],
            'answer': answers[idx],
            'score': score
        })

  return top_k_answers

In [ ]:
column_names = ['Model Name', 'Num Epochs', 'Batch Size', 'Learning Rate', 'Warmup Steps', 'Weight Decay', 'User Query', 'Relevant Question', 'Cosine Score']
output_df = pd.DataFrame(columns=column_names)

In [ ]:
def save_model_info_as_df(model_name, num_epochs, batch_size, warmup_steps, weight_decay, learning_rate, user_questions):

  global output_df

  for query in user_questions:
    top_k_answers = retrieve_top_k_answers(query)
    for answer_info in top_k_answers:
        data_to_append = {
                'Model Name': model_name,
                'Num Epochs': num_epochs,
                'Batch Size': batch_size,
                'Warmup Steps': warmup_steps,
                'Weight Decay': weight_decay,
                'Learning Rate': learning_rate,
                'User Query': query,
                'Relevant Question': answer_info['question'],
                'Cosine Score': answer_info['score']
            }

        df_new_data = pd.DataFrame([data_to_append])
        output_df = pd.concat([output_df, df_new_data], ignore_index=True)

In [ ]:
# Configure fine-tuning parameters
training_parameters = {
    'num_epochs': [2,3,4],
    'batch_size': [8,16,32],
    'warmup_steps': [100,500,1000],
    'weight_decay': [0.01,0.05,0.1],
    'learning_rate': [1e-5,3e-5,5e-5]
}

train_examples = [InputExample(texts=[faq['title'], faq['body']]) for faq in articles]

param_combinations = list(itertools.product(*training_parameters.values()))

for i,(num_epochs, batch_size, warmup_steps, weight_decay, learning_rate) in enumerate(param_combinations):
  print(f"Training with parameters: num_epochs={num_epochs}, batch_size={batch_size}, warmup_steps={warmup_steps}, weight_decay={weight_decay}, learning_rate={learning_rate}")

  model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')
  model_name = f"model_{i}"

  train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)
  train_loss = losses.MultipleNegativesRankingLoss(model)

  # Train the model
  model.fit(
      train_objectives=[(train_dataloader, train_loss)],
      epochs=num_epochs,
      warmup_steps=warmup_steps,
      weight_decay=weight_decay,
      optimizer_params={'lr': learning_rate}  # Learning rate for fine-tuning
  )

  question_embeddings = model.encode(questions, convert_to_tensor=True)

  # model_path = './drive/MyDrive/NewModel'
  # model.save(model_path)

  save_model_info_as_df(model_name, num_epochs, batch_size, warmup_steps, weight_decay, learning_rate, irrelevant_questions)

In [ ]:
output_df

In [ ]:
from google.colab import files

files.download(model_path)

output_df.to_excel('model_summaries.xlsx', index=False)
files.download('model_summaries.xlsx')